In [7]:
import cv2
from ultralytics import YOLO



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/ryanrosa/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [10]:
model = YOLO("yolov8n-pose.pt")
results = model("/Users/ryanrosa/Downloads/AI_DS_Project_Repos/ML_Movement_Tracking/Jerumelo_1.mov", save=True)


WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/815) /Users/ryanrosa/Downloads/AI_DS_Project_Repos/ML_Movement_Tracking/Jerumelo_1.mov: 384x640 5 persons, 60.8ms
video 1/1 (frame 2/815) /Users/ryanrosa/Downloads/AI_DS_Project_Repos/ML_Movement_Tracking/Jerumelo_1.mov: 384x640 5 persons, 40.1ms
video 1/1 (frame 3/815) /Users/ryanrosa/Downloads/AI_DS_Project_Repos/ML_Movement_Tracking/Jerumelo_1.mov: 384x640 5 persons, 37.7ms
video 1/1 (frame 4/815) /Users/ryanrosa/Downloads/AI_DS_Pro

In [20]:
from ultralytics import YOLO

model = YOLO("yolov8n-pose.pt")

video_path = "/Users/ryanrosa/Downloads/AI_DS_Project_Repos/ML_Movement_Tracking/Jerumelo_1.mov"

results = model.track(
    source = video_path,
    stream = True,
    classes = [0],
    tracker = 'bytetrack.yaml',
    persist = True
)



In [24]:
import cv2
import numpy as np
from ultralytics import YOLO

model = YOLO("yolov8n-pose.pt")
video_path = "/Users/ryanrosa/Downloads/AI_DS_Project_Repos/ML_Movement_Tracking/J2_vs_Mem_Martins_2.mov"

cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cx, cy = width / 2, height / 2

target_id = None

for frame in model.track(
    source = video_path,
    classes = [0],
    tracker = 'bytetrack.yaml',
    persist = True,
    stream = True
) :
    im = frame.orig_img
    boxes = frame.boxes

    if boxes is None or boxes.id is None:
        cv2.imshow("track", im)
        if cv2.waitKey(1) == 27:
            break
        continue

    ids = boxes.id.cpu().numpy()
    xyxy = boxes.xyxy.cpu().numpy()

    # Step 1: Choose Target ID
    if target_id is None and len(ids) > 0:
        #score by distance to image center and big area
        scores =[]
        for (x1, y1, x2, y2) in xyxy:
            area = (x2 -x1) * (y2 - y1)
            mx, my = (x1 + x2) / 2, (y1 + y2) / 2
            dist = np.hypot(mx - cx, my - cy)
            scores.append(area - 0.5 * dist)

        best_idx = int(np.argmax(scores))
        target_id = ids[best_idx]
        print("Chosen Target_ID:", target_id)

    #Step 2: Process the chosen target
    for i, tid in enumerate(ids):
        x1, y1, x2, y2 = xyxy[i].astype(int)

        if tid == target_id:
            #highlight target
            cv2.rectangle(im, (x1, y1), (x2, y2), (0, 255, 0), 3)
            cv2.putText(im, f"ID {int(tid)}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            #if using the pose mode, get the keypoints
            if frame.keypoints is not None:
                kpts = frame.keypoints.xy[i].cpu().numpy()
                for (kx, ky) in kpts:
                    cv2.circle(im, (int(kx), int(ky)), 3, (255, 0, 0), -1)

    cv2.imshow("track", im)
    if cv2.waitKey (1) == 27:
        break

cap.release()
cv2.destroyAllWindows()




video 1/1 (frame 1/387) /Users/ryanrosa/Downloads/AI_DS_Project_Repos/ML_Movement_Tracking/J2_vs_Mem_Martins_2.mov: 352x640 (no detections), 57.1ms
video 1/1 (frame 2/387) /Users/ryanrosa/Downloads/AI_DS_Project_Repos/ML_Movement_Tracking/J2_vs_Mem_Martins_2.mov: 352x640 (no detections), 60.0ms
video 1/1 (frame 3/387) /Users/ryanrosa/Downloads/AI_DS_Project_Repos/ML_Movement_Tracking/J2_vs_Mem_Martins_2.mov: 352x640 (no detections), 31.6ms
video 1/1 (frame 4/387) /Users/ryanrosa/Downloads/AI_DS_Project_Repos/ML_Movement_Tracking/J2_vs_Mem_Martins_2.mov: 352x640 (no detections), 39.4ms
video 1/1 (frame 5/387) /Users/ryanrosa/Downloads/AI_DS_Project_Repos/ML_Movement_Tracking/J2_vs_Mem_Martins_2.mov: 352x640 (no detections), 40.6ms
video 1/1 (frame 6/387) /Users/ryanrosa/Downloads/AI_DS_Project_Repos/ML_Movement_Tracking/J2_vs_Mem_Martins_2.mov: 352x640 (no detections), 35.4ms
video 1/1 (frame 7/387) /Users/ryanrosa/Downloads/AI_DS_Project_Repos/ML_Movement_Tracking/J2_vs_Mem_Martins_2.

In [2]:
import cv2 
import numpy as np
from ultralytics import YOLO

VIDEO_PATH = "/Users/ryanrosa/Downloads/AI_DS_Project_Repos/ML_Movement_Tracking/IMG_6022.MOV"
MODEL_PATH = 'templates/yolov8n-pose.pt'

IOU_KEEP_THRESH = 0.20 # if best IOU drops below this the track is lost

model = YOLO(MODEL_PATH)

clicked_pt = {"xy": None}

def on_mouse(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        clicked_pt["xy"] = (x, y)

def iou(a, b):
    xA, yA = max(a[0], b[0]), max(a[1], b[1])
    xB, yB = min(a,[2], b[2]), min(a[3], b[3])
    inter = max(0, xB - xA) * max(0, yB - yA)
    if inter <= 0:
        return 0.0
    areaA = (a[2] - a[0]) * (a[3] - a[1])
    areaB = (b[2] - b[0]) * (b[3] - b[1])
    return inter / (areaA + areaB - inter + 1e-9)

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise RuntimeError(f"Could not open video: {VIDEO_PATH}")

win = "Select athlete (click), press 'n' next frame, ESC quit"
cv2.namedWindow(win, cv2.WINDOW_NORMAL)
cv2.setMouseCallback(win, on_mouse)

target_box = None #xyxy float
target_kpts = None

#Selection

while True:
    ret, frame = cap.read()
    if not ret:
        raise RuntimeError("Reached end of video before selecting a target.")

    res = model.predict(frame, classes =[0], verbose = False)[0] #person
    xyxy = res.boxes.xyxy.cpu().numpy() if res.boxes is not None else np.zeros((0,4), dytpe = float)

    vis = frame.copy()
    for b in xyxy.astype(int):
        cv2.rectangle(vis, (b[0], b[1]), (b[2], b[3]), (0, 255, 255), 2)

    cv2.putText(vis, "Click athlete, then press any key. 'n' skips frame.",
                (20,30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    cv2.imshow(win, vis)

    key = cv2.waitKey(0) & 0xFF
    if key == 27:
        cap.release()
        cv2.destroyAllWindows()
        raise SystemExit

    if key == ord("n"):
        clicked_pt["xy"] = None
        continue

    if clicked_pt["xy"] is None:
        continue

    x, y = clicked_pt["xy"]
    clicked_pt["xy"] = None

    inside = []
    for i, (x1, y1, x2, y2) in enumerate(xyxy):
        if x1 <= x <= x2 and y1 <= y <= y2:
            inside.append(i)

    if not inside:
        continue

    #if a mouse click has multiple boxes, choose the smallest area
    areas = [(xyxy[i][2] -xyxy[i][0]) * (xyxy[i][3] - xyxy[i][1]) for i in inside]
    idx = inside[int(np.argmin(areas))]
    target_box = xyxy[idx]
    print("Select target box:", target_box)
    break

# Tracking Phase
while True:
    ret, frame = cap.read()
    if not ret:
        break

    res = model.predict(frame, classes = [0], verbose=False)[0]
    xyxy = res.boxes.xyxy.cpu().numpy() if res.boxes is not None else np.zeros((0,4), dtype=float)

    vis = frame.copy()

    if len(xyxy) == 0 or target_box is None:
        cv2.putText(vis, "No detections / target lost. Press 'r' to reselect",
                    (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    else:
        ious = np.array([iou(target_box, b) for b in xyxy], dtype=float)
        best_idx = int(np.argmax(ious))
        best_iou = float(ious[best_idx])

        if best_iou >= IOU_KEEP_THRESH:
            target_box = xyxy[best_idx]
            b = target_box.astype(int)
            cv2.rectanble(vis, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 3)
            cv2. putText(vis, f"Athlete (IoU {best_iou:.2f})", (b[0], max(20, b[1]-10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        else:
            cv2.putText(vis, f"Target lost (best IoU {best_iou:.2f}). Press 'r' to reselect.",
            (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    cv2.imshow(win, vis)
    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        break
    if key == ord("r"):
        target_box = None
        while target_box is None:
            res = model.predict(frame, classes=[0], verbose = False) [0]
            xyxy = res.boxes.xyxy.cpu().numpy() if res.boxes is not None else np.zeros((0, 4), dtype=float)

            vis2 = frame.copy()
            for b in xyxy.astype(int):
                cv2.rectangle(vis2, (b[0], b[1]), (b[2], b[3]), (0, 255, 255), 2)
            cv2.putText(vis2, "Reselect: click athlete, press any key. 'n' cancels reselect.",
                        (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            cv2.imshow(win, vis2)

            k2 = cv2.waitKey(0) & 0xFF
            if k2 == ord("n") or k2 == 27:
                break
            if clicked_pt["xy"] is None:
                continue

            x, y = clicked_pt["xy"]
            clicked_pt["xy"] = None
            inside = [i for i, (x1, y1, x2, y2) in enumerate(xyxy) if x1 <= x <= x2 and y1 <= y <= y2]
            if inside:
                areas = [(xyxy[i][2] - xyxy[i][0]) * (xyxy[i][3] - xyxy[i][1]) for i in inside]
                idx = inside[int(np.argmin(areas))]
                target_box = xyxy[idx]


cap.release()
cv2.destroyAllWindows()



Select target box: [     594.22      665.26      1649.7      2980.2]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()